# Cafe: 음료 판매

---

In [ ]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

from matplotlib.pylab import rcParams, style
rcParams['figure.figsize'] = 12, 8
rcParams['font.family'] = 'NanumGothic'
rcParams['font.size'] = 10

style.use('ggplot')


### DBMS 에서 데이터 읽기

In [ ]:
#import pandas.io.sql as sql
#import MySQLdb

In [ ]:
# MySQL DB에 연결
'''
con = MySQLdb.connect(host='127.0.0.1', port=3306, 
                      user=CAFE_DB_USER, 
                      passwd=CAFE_DB_PASSWD, 
                      db=CAFE_DB_NAME)
'''

In [ ]:
# SQL로 MySQL에서 가져오기
'''
orders = sql.read_sql('select * from tbl_order', conn)
order_menus = sql.read_sql('select * from tbl_ordermenu', conn)
menus = sql.read_sql('select * from tbl_menu', conn)
'''

### 데이터 로드 ( from pickle )

In [ ]:
orders      = pd.read_pickle('data/orders.pickle')
order_menus = pd.read_pickle('data/order_menus.pickle')
menus       = pd.read_pickle('data/menus.pickle')

In [ ]:
# 주문자 이름, 회사명 익명화
'''
import random_name

name_map    = dict(zip(list(orders['user'].unique()), random_name.generate(790)))  # 790
company_map = dict(zip(list(orders['company'].unique()), random_name.generate(7))) # 7

orders['user']    = orders['user'].apply(lambda x: name_map[x])
orders['company'] = orders['company'].apply(lambda x: company_map[x])
'''

In [ ]:
orders.head()

In [ ]:
menus.head()

In [ ]:
order_menus.tail()

### 데이터 분석

In [ ]:
# Merge(order_id 기준, outer join): orders + order_menus
df = orders.merge(order_menus, how='outer', left_index=True, right_on='order_id')

In [ ]:
# Merge(menu_id 기준, outer join): df + menus
df = df.merge(menus, how='outer', left_on='menu_id', right_index=True)

In [ ]:
df.head()

In [ ]:
# 결측치 확인
df[df.isnull().any(axis=1)]

#### 사용자별 주문 횟수

In [ ]:
# Grouping: 사용자 별
gp_user = df.groupby('user')

In [ ]:
# 사용자(손님)별 주문횟수
oder_cnt = gp_user.size()
oder_cnt.head()

In [ ]:
# 주문횟수로 정렬: 구매왕
oder_cnt.sort_values(inplace=True, ascending=False)

In [ ]:
oder_cnt.head()

In [ ]:
oder_cnt.describe()

#### 인덱스 재설정( 주문시간 기준)

In [ ]:
# id: index --> column
df.reset_index(inplace=True)
df.head()

In [ ]:
# created: column --> index
df.set_index('created', inplace=True)
df.head()

In [ ]:
# 인덱스 결측치 제거
df = df[df.index.notnull()]

# 2016-01-08: 샘플(테스트) 데이터 --> 제거
df = df['2016-01-09': ]

# Local time 으로 수정
df = df.shift(9, freq='H')

In [ ]:
df.head()

#### 일별 판매량

In [ ]:
# 영업일 설정(휴일 추가 등록)
from pandas.tseries.offsets import CustomBusinessDay

holidays = ['2016-02-08', '2016-02-09', '2016-02-10',
            '2016-03-01',
            '2016-03-16', '2016-03-17',
            '2016-04-13',
            '2016-04-18',
            '2016-05-05', 
            '2016-05-06'
]

bdays = CustomBusinessDay(holidays=holidays)

In [ ]:
# 일별 판매량: 영업일 기준으로 리샘플링하여 일별 카운트 계산
df['id'].resample(bdays).count().plot(figsize=(16,5))

In [ ]:
df['id'].resample(bdays).count().plot(kind='bar', figsize=(16,5))

In [ ]:
# 판매량이 140 보다 적은 날
df_t = df.resample(bdays).count()
df_t[df_t.user < 140]

### 회사별 판매량

In [ ]:
df['company'].unique()

In [ ]:
gp_com = df.groupby('company')
com_cnt = gp_com.count()
com_cnt

In [ ]:
# 회사별 사용자별 판매량
fuzzy = gp_com.get_group('fuzzy-aqua-squirrel').groupby('user').size()
messy = gp_com.get_group('messy-bistre-barnacle').groupby('user').size()
zippy = gp_com.get_group('zippy-periwinkle-bloodhound').groupby('user').size()

In [ ]:
zippy.head()

### 개인컵, 아이스 음료 주문 건수

In [ ]:
gp_com.get_group('fuzzy-aqua-squirrel').resample(bdays).get_group('2016-01-14')

In [ ]:
def com_cup_ice(com):
    return gp_com.get_group(com)[['user', 'use_cup', 'is_ice']].resample(bdays).agg({'user':np.size, 'use_cup':np.sum, 'is_ice':np.sum}).reindex(['user', 'use_cup', 'is_ice'], axis=1)

In [ ]:
com_cup_ice('fuzzy-aqua-squirrel')

#com_cup_ice('messy-bistre-barnacle')
#com_cup_ice('zippy-periwinkle-bloodhound')


### 회사별, 영업일 별 판매량

In [ ]:
df_t = gp_com.resample(bdays).count()[['id','is_ice', 'use_cup']]
df_t

In [ ]:
bday_cnt = df_t['id'].unstack(level=0).fillna(0)
bday_cnt

In [ ]:
# 3개 회사 선택
bday_cnt_com = bday_cnt[['zippy-periwinkle-bloodhound', 'messy-bistre-barnacle', 'fuzzy-aqua-squirrel']]
bday_cnt_com

In [ ]:
bday_cnt_com.plot(figsize=(16,5), kind='area')

### 회사별, 영업일 별 개인컵, 아이스 음료 판매량

In [ ]:
ice_per_com = gp_com.resample(bdays).sum()[['is_ice', 'use_cup']]
ice_per_com

In [ ]:
ice_per_com.loc['zippy-periwinkle-bloodhound']

In [ ]:
# 개인컵, 아이스 음료 판매 비율

fig, axes = plt.subplots(3,1)

def cup_n_ice(com, ax):
    ax.set_title(com)
    
    df_t = ice_per_com.loc[com]
    
    df_t['total'] = bday_cnt[com]    # 영업일별 전체 판매량
    df_t['cup_ratio']   = df_t['use_cup'] * 100 / df_t['total']
    df_t['ice_ratio']   = df_t['is_ice']  * 100 / df_t['total']
    
    df_t[['cup_ratio', 'ice_ratio']].plot(figsize=(16,8), ax=ax, sharex=True)

plt.ylim(0, 100)

cup_n_ice('zippy-periwinkle-bloodhound', axes[0])
cup_n_ice('messy-bistre-barnacle',       axes[1])
cup_n_ice('fuzzy-aqua-squirrel',         axes[2])

plt.tight_layout()

### 요일별 판매량

In [ ]:
# 요일별 판매량
df_wday = df.groupby(df.index.weekday).agg({'id': np.size})

df_wday.plot(kind='bar', figsize=(8,3))

plt.ylim(3070,3300)

### 시간대별 판매량

In [ ]:
# 시간대별 판매량
df_hour = df.groupby(df.index.hour).agg({'id': np.size}).loc[8:20]

df_hour.plot(kind='bar', figsize=(12,5))

In [ ]:
# end of file